In [1]:
import sys, os
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
from mvesc_utility_functions import *
import pandas as pd

In [2]:
import re
import numpy as np

In [3]:
import pickle

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [26]:
def get_specific_cross_tabs(crosstabs, key, feature):
    """
    Given the dictionary of all possible crosstabs, and a filename for a
    single model, and a single feature, and whether to use the predictions on
    train/val/test set, returns a nicely formatted crosstab dataframe for that
    model/feature/set.

    :param dict(tuple: list(dataframe)) crosstabs: the full crosstabs dictionary
        as computed by loop_through_top_models function
    :param str model_name: a string containing the model type for the
        specific model run to pull crosstabs for
    :param str label: a string containing the outcome label to pull
    :param str feature: a string containing the feature e.g. 'days_present_gr_9'
    :param str split: whether to pull predictions on train, val, or test
    :returns dataframe containing actual vs predicted vs marginal proportions
        on positive/negative class by binned level of feature
    :rtype pd.dataframe
    """
    crosstab = crosstabs[key][feature]
    totals = crosstab.ix['true_label: All']
    predicted = 100*crosstab.ix[['predicted_label: True',
        'predicted_label: False']]/totals
    actual = 100*crosstab.ix[['true_label: True',
        'true_label: False']]/totals
    full = predicted.append(actual)
    full = full.append(totals*100)
    full = full.round(2)
    return full

In [6]:
def build_temp_table_best_models(cursor, optimization_criteria,
    table_name = 'top_models', feature_categories = 'all',
    prediction_grade = 10, timestamp_filter = '2016-08-09 06:00:00'):
    """
    For each model_name and label category, filtered on specified options
    grab the highest ranking model according to the optimization criteria.
    Return a list of tuples containing the filename, model_name, label,
    feature list, and grade range for the models you want to pull predictions

    :param pscyopg2.cursor cursor: a cursor to execute queries on the database
    :param str table_name: name for the temporary table to hold best models
    :param str feature_categories: specify what features are given to the models
        being compared, by default include only models trained on all features
    :param int prediction_grade: the prediction time point for all models
    :param str timestamp_filter: a timestamp format string to ignore all models
        run prior to the time specified here

    :returns a list of the top models to compare features for
        each model specified as (filename, model_name, label, features, grades)
    :rtype list(tuple)
    """
    if (feature_categories == 'all'):
        feature_categories = '%,%,%'
    top_models_query = """
    drop table if exists {table};
    create temporary table {table} as
    select distinct on (model_name, label) * from
        (select model_name, filename, label, feature_categories,
        feature_grades, {criteria},
        rank() over (partition by (model_name, label)
        order by {ranker} desc) as val_rank
        from model.reports
        where debug=false
        and feature_categories like '{features}'
        and cv_scheme = 'leave_cohort_out'
        and prediction_grade = {grade}
        and timestamp > '{timestamp}'
        order by model_name, label, val_rank) vr
    order by model_name, label, val_rank;
    """.format(table = table_name, features = feature_categories,
        grade = prediction_grade, timestamp = timestamp_filter,
        criteria = ", ".join(optimization_criteria),
        ranker = optimization_criteria[0])

    cursor.execute(top_models_query)
    cursor.execute("""select filename, model_name, label,
        feature_categories, feature_grades from {table}
        """.format(table=table_name))
    models_and_features = cursor.fetchall()

    print('done grabbing models')
    return models_and_features

In [27]:
def loop_through_top_models(cursor, models_and_features,
    splits = ['train', 'val', 'test']):
    """
    Get a list of models to search over and then for each model, pull the
    predictions and feature values for every student in the specified splits
    for all features included in the feature categories and grade range of the
    model. Then bin those features into categorical variables and build rough
    crosstabs on each model-feature-split combination.

    :param list(str) optimization_criteria: a list of model selection criteria
        to compare models, with the first element used to pull the top model
    :param list(str) splits: a list of what sets should be included, i.e. some
        combination of 'train', 'test', 'val'
    :returns a dictionary of rough crosstabs, keyed by model
    """
    crosstabs_by_model_and_feature = dict()

    for (table_name, model_name, label, feature_tables,
            feature_grade_range) in models_and_features:
        print('working on {}:{}'.format(model_name, label))
        feature_table_list = feature_tables.split(", ")
        feature_grades = [int(i) for i in
                feature_grade_range.split(", ")]
        feature_grade_regex = '({})'.format('|'.join(
            [str(i) for i in feature_grades]))
        pattern = re.compile('(_gr_{rx}\Z)|(\D\Z)'.format(
            rx=feature_grade_regex))

        for test_set in splits:
            get_model_predictions = """
            select * from
            (select student_lookup, true_label, predicted_label,
            predicted_label = true_label as correct
            from predictions."{table}"
            where split = '{test_set}') preds
            """.format(table = table_name, test_set = test_set)

            for features in feature_table_list:
                get_model_predictions += """ left join
            (select * from model.{features}) {features}
            using(student_lookup)
            """.format(features = features)

            cursor.execute(get_model_predictions)
            predictions_and_features = cursor.fetchall()
            colnames = [i[0] for i in cursor.description]

            predictions = pd.DataFrame.from_records(
                predictions_and_features,
                index = 'student_lookup',
                columns = colnames)
            predictions = predictions.filter(regex=pattern)
            predictions[['true_label', 'predicted_label']] = \
            predictions[['true_label', 'predicted_label']].astype(bool)
            predictions = make_df_categorical(predictions)

            crosstabs = build_crosstabs(predictions)
            key = (table_name, model_name, label, test_set)
            crosstabs_by_model_and_feature[key] = crosstabs
    return crosstabs_by_model_and_feature

In [8]:
def build_crosstabs(prediction_data):
    # base_rates = {col: prediction_data[col].value_counts()
    #                    for col in prediction_data.columns}
    predicted = {col: pd.crosstab(index=prediction_data.predicted_label,
                    columns = prediction_data[col], margins=True,
                    normalize = True)
                for col in prediction_data.columns[3:]}
    actual = {col: pd.crosstab(index=prediction_data.true_label,
                    columns = prediction_data[col], margins=True,
                    normalize = True)
                 for col in prediction_data.columns[3:]}
    correct = {col: pd.crosstab(index=prediction_data.correct,
                    columns = prediction_data[col], margins=True,
                    normalize = True)
                 for col in prediction_data.columns[3:]}

    predicted_plus_actual = dict()
    for feature in predicted.keys():
        preds = predicted[feature]
        true = actual[feature]
        corr = correct[feature]
        preds.index = ['{name}: {value}'.format(name=preds.index.name,
                        value=value) for value in preds.index]
        true.index = ['{name}: {value}'.format(name=true.index.name,
                        value=value) for value in true.index]
        corr.index = ['{name}: {value}'.format(name=corr.index.name,
                        value=value) for value in corr.index]
        full = preds.append(true)
        full = full.append(corr)
        predicted_plus_actual[feature] = full
    return predicted_plus_actual

In [9]:
def make_df_categorical(raw_data):
    string_features = raw_data.select_dtypes(include=[object, bool])
    numeric_features = raw_data.select_dtypes(include=[np.number])

    for string_col in string_features.columns:
        raw_data[string_col] = string_features[string_col].astype('category')
        if (len(raw_data[string_col].cat.categories) < 2
            and string_col != 'predicted_label'
            and string_col != 'true_label'
            and string_col != 'correct'):
            raw_data.drop(string_col, axis=1, inplace=True)

    for numeric_col in numeric_features.columns:
        num_values = len(numeric_features[numeric_col].unique())
        num_bins = min(5, num_values)
        if (num_values < 2):
            raw_data.drop(numeric_col, axis=1, inplace=True)
        elif (num_values == 2 and
            numeric_features[numeric_col].isnull().sum() > 0):
            raw_data[numeric_col] = numeric_features[numeric_col] \
                .isnull().astype('category')
        else:
            raw_data[numeric_col] = pd.cut(numeric_features[numeric_col],
                bins = num_bins, precision = 1)

    return raw_data

In [10]:
path = os.path.join('/home/jgutman/mvesc', 'Error_Feature_Analysis', 'pkls')
filename = 'updated_crosstabs.pkl'

In [11]:
# load pkl file
pkl_file = open(os.path.join(path, filename),'rb')
all_top_crosstabs = pickle.load(pkl_file)

In [12]:
model_list = [model[0] for model in all_top_crosstabs.keys()]

In [28]:
load_pkl = False

In [29]:
filename = 'selected_RF_not_on_time.pkl'

In [30]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        select_hanna_models = """
    select distinct on (filename) filename, model_name, label, feature_categories, 
    feature_grades from model.reports
    where filename in ('08_09_2016_grade_6_param_set_0_RF_ht_8585', 
    '08_09_2016_grade_7_param_set_7_RF_ht_10497', '08_09_2016_grade_8_param_set_13_RF_ht_13254',
    '08_09_2016_grade_9_param_set_0_RF_ht_8645', '08_09_2016_grade_10_param_set_0_RF_ht_8680')
    ;"""
        cursor.execute(select_hanna_models)
        models_and_features = cursor.fetchall()

In [31]:
# either load a stored pkl or compute it from scratch
if load_pkl:
    # load pkl file
    pkl_file = open(os.path.join(path, filename),'rb')
    all_top_crosstabs = pickle.load(pkl_file)
else:
    with postgres_pgconnection_generator() as connection:
        with connection.cursor() as cursor:
            #models_and_features = build_temp_table_best_models(
            #    cursor, optimization_criteria)
            all_top_crosstabs = loop_through_top_models(
                cursor, models_and_features, ['val', 'test'])
            with open(os.path.join(path, filename), 'wb') as f:
                pickle.dump(all_top_crosstabs, f)

working on RF:not_on_time
working on RF:not_on_time
working on RF:not_on_time
working on RF:not_on_time
working on RF:not_on_time


In [33]:
len(all_top_crosstabs.keys())

10

In [40]:
# models and features to display crosstabs for
ignore_splits = set('train')
ignore_models = set()
ignore_outcomes = set()
specific_feature_list = dict()

In [ ]:
for model in all_top_crosstabs.keys():
    filename = model[0]
    model_name = model[1]
    outcome = model[2]
    split = model[3]
    if (model_name in ignore_models or split in ignore_splits or
        outcome in ignore_outcomes):
        continue
    else:
        if not specific_feature_list:
            feature_list = all_top_crosstabs[model].keys()
        else:
            feature_list = specific_feature_list
        for feature in feature_list:
            try:
                print(filename, model_name, outcome, split)
                get_specific_cross_tabs(
                    all_top_crosstabs, model, feature)
            except KeyError:
                print('{} not in {}'.format(feature, filename))
                continue

08_09_2016_grade_9_param_set_0_RF_ht_8645 RF not_on_time val


tardy_wkd_1_gr_8,"(-0.007, 1.4]","(1.4, 2.8]","(2.8, 4.2]","(4.2, 5.6]","(5.6, 7]",All
predicted_label: True,17.74,20.00,25.00,25.00,0.00,17.83
predicted_label: False,82.26,80.00,75.00,75.00,100.00,82.17
true_label: True,25.45,40.00,45.83,75.00,0.00,25.88
true_label: False,74.55,60.00,54.17,25.00,100.00,74.12
true_label: All,97.61,1.19,0.96,0.16,0.08,100.00


08_09_2016_grade_9_param_set_0_RF_ht_8645 RF not_on_time val


tardy_wkd_2_gr_8,"(-0.01, 2.4]","(2.4, 4.8]","(4.8, 7.2]","(7.2, 9.6]","(9.6, 12]",All
predicted_label: True,17.85,15.0,0.00,NaN,100.00,17.83
predicted_label: False,82.15,85.0,100.00,NaN,0.00,82.17
true_label: True,25.76,35.0,33.33,NaN,100.00,25.88
true_label: False,74.24,65.0,66.67,NaN,0.00,74.12
true_label: All,99.04,0.8,0.12,0.0,0.04,100.00


08_09_2016_grade_9_param_set_0_RF_ht_8645 RF not_on_time val


...

08_09_2016_grade_8_param_set_13_RF_ht_13254 RF not_on_time val


days_absent_excused_gr_6,"(-0.08, 15.3]","(15.3, 30.6]","(30.6, 45.9]","(45.9, 61.2]","(61.2, 76.5]",All
predicted_label: True,10.06,12.99,0.00,0.00,0.00,10.19
predicted_label: False,89.94,87.01,100.00,100.00,100.00,89.81
true_label: True,20.80,28.57,50.00,100.00,0.00,21.31
true_label: False,79.20,71.43,50.00,0.00,100.00,78.69
true_label: All,94.23,5.49,0.14,0.07,0.07,100.00


08_09_2016_grade_8_param_set_13_RF_ht_13254 RF not_on_time val


days_present_gr_7,"(-0.2, 36]","(36, 72]","(72, 108]","(108, 144]","(144, 180]",All
predicted_label: True,25.00,7.14,20.00,16.67,9.76,10.19
predicted_label: False,75.00,92.86,80.00,83.33,90.24,89.81
true_label: True,43.75,35.71,46.67,63.89,19.44,21.31
true_label: False,56.25,64.29,53.33,36.11,80.56,78.69
true_label: All,1.14,1.00,1.07,2.57,94.23,100.00


08_09_2016_grade_8_param_set_13_RF_ht_13254 RF not_on_time val
